In [48]:
import os
os.chdir("/home/icb/florin.ratajczak/ppi-core-genes/")
from speos.preprocessing.preprocessor import PreProcessor
from speos.preprocessing.mappers import *
from speos.utils.config import Config

In [75]:
import pandas as pd

methods = ["film","film_unbiased", "tag", "gcn", "n2vmlp", "mlp"]
pretty_methods = ["FiLM", "FiLM-Unbiased", "TAG", "GCN", "N2V+MLP", "MLP"]

diseases = ["cardiovascular", "immune_dysregulation", "bm_disorder", "insulin_disorder", "diabetes"]
pretty_diseases = ["Cardiovascular_Disease", "Immune_Dysregulation", "Body_Mass_Disorders", "Insulin_Disorders", "Diabetes"]
tags = ["Cardiovascular_Disease","Immune_Dysregulation","Body_Mass_Disorder","Insulin_Disorder", "Monogenic_Diabetes"]

canonical_order = pd.read_csv("data/hgnc_official_list.tsv", header=0, sep="\t", usecols=["symbol"])["symbol"].tolist()
order_dict = {name: value for value, name in enumerate(canonical_order)}

for disease, pretty_disease, tag in zip(diseases, pretty_diseases, tags):

    df = None
    true_mendelians = set(pd.read_csv("/home/icb/florin.ratajczak/ppi-core-genes/data/mendelian_gene_sets/{}_genes.bed".format(tag),header=None, sep="\t", names=["Chr", "start", "stop", "symbol", "strand"], usecols=["symbol"])["symbol"].tolist())

    for method, pretty_method in zip(methods[::-1], pretty_methods[::-1]):
        if method == "film" and disease == "immune_dysregulation":
            method = "film_forreal"
        if method in ["gcn", "n2vmlp", "mlp"] and disease == "immune_dysregulation":
            method += "_new"
        


        new_df = pd.read_csv("/lustre/groups/epigenereg01/projects/ppi-florin/results/{}_{}_pp_table.tsv".format(disease, method), header=0, sep="\t", index_col=0)
        new_df = new_df.sort_index(key=lambda x: x.map(order_dict))
        new_df.columns = [column + " " + pretty_method if column == "CS" else column for column in new_df.columns]

        true_mendelians = set(new_df.index.tolist()).intersection(true_mendelians)

        mendelian_hgncs_in_df = set(new_df.index[new_df["Mendelian"]].tolist())

        if len(true_mendelians) != len(mendelian_hgncs_in_df):
            # check that all mendelians in the df are included in the total mendelians
            assert len(true_mendelians.intersection(mendelian_hgncs_in_df)) == len(mendelian_hgncs_in_df)

            # assign all labels to df to get mendelians that are missing in the pp table
            new_df.loc[true_mendelians, ["Mendelian"]] = True

            mendelian_hgncs_in_df = set(new_df.index[new_df["Mendelian"]].tolist())

            # check that all mendelians in the df are included in the total mendelians
            assert len(true_mendelians.intersection(mendelian_hgncs_in_df)) == len(true_mendelians) == len(mendelian_hgncs_in_df)

        if df is None:
            new_df.drop("Candidate", axis=1, inplace=True)
            df = new_df
            
        else:
            grab_columns = [column if column.split(" ")[0] == "CS" else column for column in new_df.columns]
            for grab_column in grab_columns:
                df[grab_column] = new_df[grab_column]
            new_columns = df.columns[:2].tolist() + [df.columns[-1]] + df.columns[2:-1].tolist()
            df = df[new_columns]

    df.drop("DGE: Union", axis=1, inplace=True)
    df.drop("Candidate", axis=1, inplace=True)
    df.drop("pLI>0.9", axis=1, inplace=True)
    lof_df = pd.read_csv("data/forweb_cleaned_exac_r03_march16_z_data_pLI.txt", index_col=0, header= 0, sep="\t", usecols=["gene", "mis_z", "lof_z"])
    lof_df.columns = ["Missense Z-Score", "LoF Z-Score"]
    df = df.join(lof_df, how="left")
    df = df.rename_axis('HGNC').reset_index()
    for column in df.columns:
        if column.startswith("CS") or column == "Number of Drug Interactions":
            df[column] = df[column].astype(int)
    df.to_csv("results/master_tables/{}_master_table.csv".format(pretty_disease), sep="\t", index=False)

        

In [77]:
len(true_mendelians.intersection(mendelian_hgncs_in_df))

182

In [78]:
len(true_mendelians)

182

In [79]:
len(mendelian_hgncs_in_df)

182

In [81]:
new_df["Mendelian"][new_df["Is Included"]].sum()

179

In [62]:
mendelians = pd.read_csv("/home/icb/florin.ratajczak/ppi-core-genes/data/mendelian_gene_sets/Cardiovascular_Disease_genes.bed",header=None, sep="\t", names=["Chr", "start", "stop", "symbol", "strand"], usecols=["symbol"])["symbol"].tolist()

In [43]:
import pandas as pd
canonical_order = pd.read_csv("../data/hgnc_official_list.tsv", header=0, sep="\t", usecols=["symbol"])["symbol"].tolist()

new_df = pd.read_csv("/lustre/groups/epigenereg01/projects/ppi-florin/results/cardiovascular_film_pp_table.tsv", header=0, sep="\t", index_col=0)

In [2]:
new_df.head()

,Is Included,Mendelian,Candidate,CS,DGE: Coronary Artery Disease,DGE: Atrial Fibrillation,DGE: Aortic Aneurysm,DGE: Ischemia,DGE: Hypertension,DGE: Artherosclerosis,DGE: Union,Drug Target,Number of Drug Interactions,Druggable,Included in Mouse KO,Is Mouse KO,pLI>0.9
A1BG,True,False,True,9.0,False,False,False,False,False,False,False,True,5.0,True,True,False,False
A1CF,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,False
A2M,True,False,False,0.0,False,False,False,False,False,False,False,True,29.0,True,True,False,False
A2ML1,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,True,False,False,False
A3GALT2,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,False


In [44]:
order_dict = {name: value for value, name in enumerate(canonical_order)}
sorted_df = new_df.sort_index(key=lambda x: x.map(order_dict))


In [46]:
sorted_df.index.tolist()

['A1BG',
 'A1CF',
 'A2M',
 'A2ML1',
 'A3GALT2',
 'A4GALT',
 'A4GNT',
 'AAAS',
 'AACS',
 'AADAC',
 'AADACL2',
 'AADACL3',
 'AADACL4',
 'AADAT',
 'AAGAB',
 'AAK1',
 'AAMDC',
 'AAMP',
 'AANAT',
 'AAR2',
 'AARD',
 'AARS1',
 'AARS2',
 'AARSD1',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCA10',
 'ABCA12',
 'ABCA13',
 'ABCB1',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCB10',
 'ABCB11',
 'ABCC1',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD16A',
 'ABHD16B',
 'ABHD17A',
 'ABHD17B',
 'ABHD17C',
 'ABHD18',
 'ABI1',
 '

In [17]:
new_df.loc["MTOR"]

Is Included                      True
Mendelian                       False
Candidate                       False
CS                                0.0
DGE: Coronary Artery Disease    False
DGE: Atrial Fibrillation        False
DGE: Aortic Aneurysm            False
DGE: Ischemia                   False
DGE: Hypertension                True
DGE: Artherosclerosis           False
DGE: Union                       True
Drug Target                      True
Number of Drug Interactions     208.0
Druggable                        True
Included in Mouse KO             True
Is Mouse KO                      True
pLI>0.9                          True
Name: MTOR, dtype: object

In [34]:
lof_df = pd.read_csv("../data/forweb_cleaned_exac_r03_march16_z_data_pLI.txt", index_col=0, header= 0, sep="\t", usecols=["gene", "mis_z", "lof_z"])
lof_df.columns = ["Missense Z-Score", "LoF Z-Score"]

In [35]:
lof_df

,Missense Z-Score,LoF Z-Score
gene,,
AGRN,0.140544,5.561613
NOC2L,-2.275892,-0.021471
B3GALT6,3.209371,0.572158
C1orf159,-0.419916,1.130742
ISG15,0.212786,0.269984
...,...,...
TMSB4Y,-1.057141,0.513697
DDX3Y,1.417526,2.134305
NLGN4Y,1.070605,1.739507


In [36]:
joined_df = sorted_df.join(lof_df, how="left")

In [37]:
joined_df

,Is Included,Mendelian,Candidate,CS,DGE: Coronary Artery Disease,DGE: Atrial Fibrillation,DGE: Aortic Aneurysm,DGE: Ischemia,DGE: Hypertension,DGE: Artherosclerosis,DGE: Union,Drug Target,Number of Drug Interactions,Druggable,Included in Mouse KO,Is Mouse KO,pLI>0.9,Missense Z-Score,LoF Z-Score
A1BG,True,False,True,9.0,False,False,False,False,False,False,False,True,5.0,True,True,False,False,1.031431,1.214845
A1CF,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,False,0.387458,3.294270
A2M,True,False,False,0.0,False,False,False,False,False,False,False,True,29.0,True,True,False,False,0.516917,4.918822
A2ML1,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,True,False,False,False,-0.791240,1.860684
A3GALT2,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,False,1.963897,-0.344024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,False,NaN,NaN
ZYG11B,True,False,False,0.0,False,False,False,False,False,False,False,False,0.0,False,True,False,True,3.924536,5.111337
ZYX,True,False,False,0.0,False,False,False,False,False,False,False,True,6.0,False,True,False,False,-0.578396,1.736543
ZZEF1,True,False,False,0.0,False,False,False,False,False,False,False,True,2.0,False,True,False,True,0.554271,8.465035


In [39]:
lof_df.loc["A2M"]

Missense Z-Score    0.516917
LoF Z-Score         4.918822
Name: A2M, dtype: float64